# Generate intervention scenario datasets

This notebook generates multiple *synthetic* datasets, each with a **different intervention**, all based on the same baseline SIR parameters.

Outputs (CSV):
- `data/processed/scenarios/intervention_lockdown_beta_step.csv`
- `data/processed/scenarios/intervention_treatment_gamma_step.csv`
- `data/processed/scenarios/intervention_vaccination_pulse.csv`

See `docs/SCENARIO_DATASETS.md` for definitions.


In [ ]:
from pathlib import Path

from epidemiology_project.paths import processed_data_dir
from epidemiology_project.sir import SirParams, simulate_sir, step_change


## Baseline parameters (shared across scenarios)


In [ ]:
params = SirParams(N=1_000_000, beta=0.28, gamma=0.10)
days = 200
I0 = 100
t0 = 30  # day intervention starts


## Scenario 1: Lockdown (beta step down)


In [ ]:
lockdown_beta = step_change(after_day=t0, factor=0.5)
df_lockdown = simulate_sir(
    params=params,
    I0=I0,
    days=days,
    beta_schedule=lockdown_beta,
    scenario='intervention_lockdown_beta_step',
)
df_lockdown.head()


## Scenario 2: Treatment (gamma step up)


In [ ]:
treatment_gamma = step_change(after_day=t0, factor=1.5)
df_treatment = simulate_sir(
    params=params,
    I0=I0,
    days=days,
    gamma_schedule=treatment_gamma,
    scenario='intervention_treatment_gamma_step',
)
df_treatment.head()


## Scenario 3: Vaccination pulse (move S → R at t0)


In [ ]:
df_vax = simulate_sir(
    params=params,
    I0=I0,
    days=days,
    vaccination_pulse=(t0, 0.20),  # vaccinate 20% of susceptibles at t0
    scenario='intervention_vaccination_pulse',
)
df_vax.head()


## Save datasets


In [ ]:
out_dir = processed_data_dir() / 'scenarios'
out_dir.mkdir(parents=True, exist_ok=True)

paths = {
    'intervention_lockdown_beta_step.csv': df_lockdown,
    'intervention_treatment_gamma_step.csv': df_treatment,
    'intervention_vaccination_pulse.csv': df_vax,
}
for name, df in paths.items():
    p = out_dir / name
    df.to_csv(p, index=False)
    print('Wrote', p)
